In [103]:
import pandas as pd

In [104]:
criteria_df, dimension_df, impactscore_df = pd.read_excel(
    "Criteria Score.xlsx", 
    sheet_name=["Sheet1", "Sheet2", "Sheet3"]
).values()

In [105]:
#impact on sector 
def is_text(x):
    return isinstance(x, str)
    
criteria_df['Impact on Sector'] = criteria_df['Value'].apply(is_text)

In [106]:
#amount 
def map_value_to_amount(x):
    if pd.isna(x):
        return None
    elif isinstance(x, (int, float)):
        return x
    elif isinstance(x, str):
        if x.strip().lower() == 'low impact':
            return 0
        elif x.strip().lower() == 'moderate impact':
            return 0.5
        elif x.strip().lower() == 'high impact':
            return 1
    return None  

criteria_df['Amount'] = criteria_df['Value'].apply(map_value_to_amount)

In [107]:
#min and max values
min_vals = criteria_df.groupby(['Dimensions', 'Criteria'])['Amount'].transform('min')
max_vals = criteria_df.groupby(['Dimensions', 'Criteria'])['Amount'].transform('max')

criteria_df['Actual_Min'] = min_vals
criteria_df['Actual_Max'] = max_vals

In [108]:
df

,Company Name,Dimensions,Criteria,Year,Quarter,Value,Impact on Sector,Amount,Actual_Min,Actual_Max,Scaled Amount,Weight,Criteria Score
0,HDFC Bank Ltd.,AuM,Approved Committed Capital,2024,Q1,521200,False,521200.00,111.69,521200.0,NaN,0.500000,NaN
1,HDFC Bank Ltd.,AuM,Current AuM,2024,NaN,787400,False,787400.00,6480.00,3889200.0,NaN,0.500000,NaN
2,HDFC Bank Ltd.,Micro Economic Objectives,Direct Job Creation,2024,NaN,40305,False,40305.00,-1670.00,40305.0,NaN,0.333333,NaN
3,HDFC Bank Ltd.,Micro Economic Objectives,Local content,2024,NaN,100,False,100.00,5.00,100.0,NaN,0.333333,NaN
4,HDFC Bank Ltd.,Micro Economic Objectives,NGI,2024,NaN,2025,False,2025.00,125.00,355000.0,NaN,0.333333,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,ADITYA BIRLA REAL ESTATE LIMITED,Impact on sector,Growth Rate,2024,NaN,Low Impact,True,0.00,0.00,1.0,NaN,0.250000,NaN
308,ADITYA BIRLA REAL ESTATE LIMITED,Impact on sector,Jobs and sector Localization,2024,NaN,Moderate Impact,True,0.50,0.00,1.0,NaN,0.250000,NaN
309,ADITYA BIRLA REAL ESTATE LIMITED,Impact on sector,Portfolio strategy contribution,2024,NaN,Low Impact,True,0.00,0.00,1.0,NaN,0.250000,NaN
310,ADITYA BIRLA REAL ESTATE LIMITED,Impact on sector,Role in sector,2024,NaN,Moderate Impact,True,0.50,0.50,1.0,NaN,0.250000,NaN


In [109]:
#scaled amount 

def calculate_scaled_amount(row):
    amt = row['Amount']
    min_val = row['Actual_Min']
    max_val = row['Actual_Max']

    try:
        if (max_val - min_val) == 0:
            return 1
        else:
            return (amt - min_val) / (max_val - min_val)
    except:
        return 0

criteria_df['Scaled Amount'] = criteria_df.apply(calculate_scaled_amount, axis=1)

In [110]:
#weight 
criteria_count = criteria_df.groupby('Dimensions')['Criteria'].transform('nunique')

criteria_df['Weight'] = 1.0 / criteria_count

In [111]:
criteria_df['Criteria Score'] = criteria_df['Scaled Amount'] * criteria_df['Weight']

In [112]:
#agg. criteria score
def get_aggregate_score(row):
    filtered = criteria_df[
        (criteria_df['Company Name'] == row['Company Name']) &
        (criteria_df['Dimensions'] == row['Dimensions']) &
        (criteria_df['Year'] == row['Year']) &
        (criteria_df['Quarter'] == row['Quarter'])
    ]
    return filtered['Criteria Score'].sum()

In [113]:
dimension_df['Agg. Criteria Score'] = dimension_df.apply(get_aggregate_score, axis=1)

In [114]:
#weight
dimension_counts = dimension_df.groupby('Company Name')['Dimensions'].transform('count')
dimension_df['Weight'] = 1 / dimension_counts

In [115]:
#dimension score
dimension_df['Dimension Score'] = dimension_df['Weight'] * dimension_df['Agg. Criteria Score']

In [116]:
#impact score 
def get_impact_score(row):
    filtered = dimension_df[
        (dimension_df['Company Name'] == row['Company Name']) &
        (dimension_df['Year'] == row['Year']) &
        (dimension_df['Quarter'] == row['Quarter'])
    ]
    return filtered['Dimension Score'].sum()

In [117]:
impactscore_df['Impact Score'] = impactscore_df.apply(get_impact_score, axis=1)

In [118]:
with pd.ExcelWriter("Criteria Score - Final Weighted.xlsx") as writer:
    criteria_df.to_excel(writer, sheet_name="Sheet1", index=False)
    dimension_df.to_excel(writer, sheet_name="Sheet2", index=False)
    impactscore_df.to_excel(writer, sheet_name="Sheet3", index=False)